In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from pydrake.geometry.optimization import HPolyhedron
from pydrake.trajectories import PiecewisePolynomial

from spp.bezier import BezierSPP
from spp.linear import LinearSPP

In [ ]:
from scipy.spatial import ConvexHull

def plot_vpolytope(V, **kwargs):
    plt.fill(*V.T, **kwargs)
    
def make_hpolytope(V):
    ch = ConvexHull(V)
    return HPolyhedron(ch.equations[:, :-1], - ch.equations[:, -1])

In [ ]:
from models.env_2d import obstacles, vertices

regions = [make_hpolytope(V) for V in vertices]

x_start = np.array([0, 0])
x_goal = np.array([2.4, 2.1])

In [ ]:
plt.figure()

for O in obstacles:
    plt.fill(*O.T, fc='lightcoral', ec='k', zorder=4)

plt.plot(*x_start, 'kx', markersize=10)
plt.plot(*x_goal, 'kx', markersize=10)

plt.text(.05, -.2, '$q_0$', size=16, ha='center')
plt.text(2.35, 1.9, '$q_T$', size=16, ha='center')

plt.axis('square')
x_min = np.min(np.vstack(vertices), axis=0)
x_max = np.max(np.vstack(vertices), axis=0)
plt.xlim([x_min[0], x_max[0]])
plt.ylim([x_min[1], x_max[1]])

x_ticks = np.linspace(-.1, 2.5, 27)
y_ticks = np.linspace(-.4, 2.6, 31)
plt.xticks(x_ticks)
plt.yticks(y_ticks)

keep = lambda t: np.isclose(t % .5, 0) or np.isclose(t % .5, .5)
x_labels = [round(t, 1) if keep(t) else '' for t in x_ticks]
y_labels = [round(t, 1) if keep(t) else '' for t in y_ticks]
plt.gca().set_xticklabels(x_labels)
plt.gca().set_yticklabels(y_labels)
plt.grid(1)
plt.savefig('setup.pdf', bbox_inches='tight')

In [ ]:
plt.figure()

for V in vertices:
    plot_vpolytope(V, fc='lightcyan', ec='k', zorder=4)

plt.axis('square')
plt.xlim([x_min[0], x_max[0]])
plt.ylim([x_min[1], x_max[1]])

plt.xticks(x_ticks)
plt.yticks(y_ticks)
plt.gca().set_xticklabels(x_labels)
plt.gca().set_yticklabels(y_labels)
plt.grid(1)

plt.savefig('decomposition.pdf', bbox_inches='tight')

In [ ]:
n_samples = 500
relaxation = False
velocity_limits = np.array([[[-1, -1], [1, 1]]])

def solve_linear():
    l_spp = LinearSPP(regions)
    waypoints, _, _ = l_spp.SolvePath(x_start, x_goal, relaxation)
    t_knots = np.linspace(0, 1,  waypoints.shape[1])
    l_traj = PiecewisePolynomial.FirstOrderHold(t_knots, waypoints)
    l_times = np.linspace(
        l_traj.start_time(),
        l_traj.end_time(),
        n_samples
    )
    return l_traj, l_times


def solve_bezier(order, continuity):
    weights = {'time': 1}
    b_spp = BezierSPP(regions,
                      order=order,
                      continuity=continuity,
                      weights=weights,
                      deriv_limits=velocity_limits)
    b_traj, _, _ = b_spp.SolvePath(x_start, x_goal, relaxation)
    b_times = np.linspace(
        b_traj.start_time(),
        b_traj.end_time(),
        n_samples
    )
    return b_traj, b_times

In [ ]:
def plot_trajectory(traj, times):

    plt.figure()

    for O in obstacles:
        plt.fill(*O.T, fc='lightcoral', ec='k', zorder=4)

    plt.plot(*x_start, 'kx', markersize=10)
    plt.plot(*x_goal, 'kx', markersize=10)

    plt.axis('square')
    plt.xlim([x_min[0], x_max[0]])
    plt.ylim([x_min[1], x_max[1]])

    values = np.squeeze([traj.value(t) for t in times])
    plt.plot(*values.T, 'b', linewidth=2, zorder=5)

    plt.grid(1)
    
plot_trajectory(*solve_linear())
plt.savefig('linear.pdf', bbox_inches='tight')

In [ ]:
def plot_velocity(traj, times):

    plt.figure()
    velocity = np.array([traj.EvalDerivative(t) for t in times])
    plt.plot(times, np.ones(len(times)), 'r--', label='Limit', linewidth=3)
    plt.plot(times, -np.ones(len(times)), 'r--', linewidth=3)
    plt.plot(times[1:-1], velocity[1:-1, 0], label='Horizontal')
    plt.plot(times[1:-1], velocity[1:-1, 1], label='Vertical')
    plt.legend(loc=0)
    plt.grid()
    plt.xlim([times[0], times[-1]])

In [ ]:
traj, times = solve_bezier(1, 0)

plot_trajectory(traj, times)
plt.savefig('bezier_10.pdf', bbox_inches='tight')

plot_velocity(traj, times)
plt.savefig('bezier_10_vel.pdf', bbox_inches='tight')

In [ ]:
traj, times = solve_bezier(3, 1)

plot_trajectory(traj, times)
plt.savefig('bezier_31.pdf', bbox_inches='tight')

plot_velocity(traj, times)
plt.savefig('bezier_31_vel.pdf', bbox_inches='tight')

In [ ]:
traj, times = solve_bezier(4, 2)

plot_trajectory(traj, times)
plt.savefig('bezier_42.pdf', bbox_inches='tight')

plot_velocity(traj, times)
plt.savefig('bezier_42_vel.pdf', bbox_inches='tight')